In [18]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [19]:
cnx = mysql.connector.connect(user='root', password='password', host='127.0.0.1', database='otus_db', port='3306')

In [14]:
cnx.close()
cur.close()

True

In [20]:
cur = cnx.cursor()

#### Дроп / создание таблицы customers (клиенты)
##### customer_id -  id клиента
##### email -  почта клиента
##### gender -  пол клиента
##### city -  город клиента

In [27]:
cur.execute('''
            DROP TABLE IF EXISTS customers;
            ''')

cur.execute('''
            CREATE TABLE customers (
            customer_id SERIAL PRIMARY KEY,
            email VARCHAR(255) UNIQUE NOT NULL,
            gender CHAR(1) NOT NULL,
            city VARCHAR(255)
            );
            ''')

#### Наполнение таблицы customers

In [28]:
cur.execute('''
           INSERT  INTO customers
           (email, gender, city)
            VALUES
            ('dmitry@example.com', 'М', 'Зеленоград'),
            ('vasya@example.com', 'М', 'Москва'),
            ('olga@example.com', 'Ж', 'Волгоград');
            ''')

query = f'''
            SELECT * from customers;
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,email,gender,city
0,1,dmitry@example.com,М,Зеленоград
1,2,vasya@example.com,М,Москва
2,3,olga@example.com,Ж,Волгоград


#### Дроп / создание таблицы sessions (сессии клиентов)
##### session_id - id сессии 
##### customer_id -  id клиента
##### email -  почта клиента
##### gender -  пол клиента
##### city -  город клиента

In [29]:
cur.execute('''
            DROP TABLE IF EXISTS sessions;
            ''')

cur.execute('''
            CREATE TABLE sessions (
                session_id SERIAL PRIMARY KEY,
                customer_id  BIGINT UNSIGNED NOT NULL,
                visit_dttm DATETIME NOT null,
                Purchase_flg TINYINT(1) NOT null,
            CONSTRAINT fk__sessions__customers
                FOREIGN KEY (customer_id)
                REFERENCES customers (customer_id)
            );
            ''')

#### Наполнение таблицы sessions

In [40]:
cur.execute('''
           INSERT  INTO sessions
           (customer_id, visit_dttm, Purchase_flg)
            VALUES
            ('1', now() - INTERVAL 1 DAY , 0),
            ('2', now() - INTERVAL 1 DAY, 0),
            ('3', now() - INTERVAL 1 DAY, 0),
            ('1', now(), 1),
            ('2', now(), 0),
            ('3', now(), 1);;
            ''')

query = f'''
            SELECT * from sessions;
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,session_id,customer_id,visit_dttm,Purchase_flg
0,7,1,2023-09-04 18:16:25,0
1,8,2,2023-09-04 18:16:25,0
2,9,3,2023-09-04 18:16:25,0
3,10,1,2023-09-05 18:16:25,1
4,11,2,2023-09-05 18:16:25,0
5,12,3,2023-09-05 18:16:25,1


#### inner join
##### Посмотреть данные клиентов и сессий с флагом покупки 

In [45]:
query = f'''
            SELECT * from customers l
            join sessions r
             on l.customer_id = r.customer_id
             where Purchase_flg = 1;
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,email,gender,city,session_id,customer_id,visit_dttm,Purchase_flg
0,1,dmitry@example.com,М,Зеленоград,10,1,2023-09-05 18:16:25,1
1,3,olga@example.com,Ж,Волгоград,12,3,2023-09-05 18:16:25,1


#### left join 
##### Посмотреть данные клиентов у которых нет сессий

In [ ]:
cur.execute('''
           INSERT  INTO customers
           (email, gender, city)
            VALUES
            ('denis@example.com', 'М', 'Питер');
            ''')

In [49]:
query = f'''
            SELECT * from customers l
            left join sessions r
             on l.customer_id = r.customer_id
             where session_id is null;
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,email,gender,city,session_id,customer_id,visit_dttm,Purchase_flg
0,4,denis@example.com,М,Питер,None,None,None,None


#### 5 запросов с WHERE с использованием разных операторов

#### 1) Создадим агрегат витрину с количеством покупок у клиентов и зальем в нее данные 

In [51]:
cur.execute('''
            CREATE TABLE p_sessions_count (
            customer_id BIGINT UNSIGNED NOT NULL,
            count BIGINT UNSIGNED NOT NULL,
            CONSTRAINT fk__p_sessions_count__customers
                FOREIGN KEY (customer_id)
                REFERENCES customers (customer_id)
            );
            ''')

In [56]:
cur.execute('''
            INSERT INTO p_sessions_count
            (customer_id, count)
            SELECT 
                customer_id,
                count(*)
            FROM sessions
            WHERE Purchase_flg = 1
            GROUP BY customer_id;
            ''')

query = f'''
            SELECT * from p_sessions_count
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,count
0,1,1
1,3,1


#### 2) Обновим город клиента

In [61]:
cur.execute('''
            UPDATE customers
            SET city = 'Москва'
            WHERE customer_id = 4
            ''')

query = f'''
            SELECT * from customers
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,email,gender,city
0,1,dmitry@example.com,М,Зеленоград
1,2,vasya@example.com,М,Москва
2,3,olga@example.com,Ж,Волгоград
3,4,denis@example.com,М,Москва


#### 3) Удалить клиента из таблицы

In [67]:
cur.execute('''
            DELETE FROM customers l
             where customer_id = 4;
            ''')

query = f'''
            SELECT * from customers
'''
df = pd.read_sql_query(query, cnx)
df

,customer_id,email,gender,city
0,1,dmitry@example.com,М,Зеленоград
1,2,vasya@example.com,М,Москва
2,3,olga@example.com,Ж,Волгоград


#### 4) Вывести уникальные id и email клиентов у которых дата визита меньше now() и город не Москва

In [72]:
query = f'''
            select customer_id, email
                from customers c
                join sessions s
                using(customer_id)
                where visit_dttm < now() 
                and city not in ('Москва')
                group by customer_id, email
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,email
0,1,dmitry@example.com
1,3,olga@example.com


In [82]:
query = f'''
              select * 
            from sessions
            where Purchase_flg = 1
            ORDER BY customer_id desc
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,session_id,customer_id,visit_dttm,Purchase_flg
0,12,3,2023-09-05 18:16:25,1
1,10,1,2023-09-05 18:16:25,1


##### 5) Вывести данные по предпоследнему id клиента с флагом покупки 

In [88]:
query = f'''
            select * 
            from sessions
            where Purchase_flg = 1
            ORDER BY customer_id desc
            LIMIT 1, 1;
'''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,session_id,customer_id,visit_dttm,Purchase_flg
0,10,1,2023-09-05 18:16:25,1
